In [3]:
from __future__ import print_function, division

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms, utils
import time
import copy

import warnings
warnings.filterwarnings('ignore')

plt.ion()

In [2]:
#data loading
data_dir = 'data/GTSRB'

In [5]:
#resizing dataset to 32x32, meansubstraction (Normalize???)

data_transforms = {
    'Training': transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [1, 1, 1])
    ]),
    'Test': transforms.Compose([
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [1, 1, 1])
    ]),
    
}

In [14]:
class Resize(object):
    def __init__(self, w, h):
        self.w = w
        self.h = h
    def __call__(self, image):
        return transform.resize(
        image,
        output_shape = (self.w, self.h),
        preserve_range = True,
        mode = "constant"
        )
    
class SubtractMean(object):
    def __call__(self, image):
        ch1 = image[:, :, 0]
        ch2 = image[:, :, 1]
        ch3 = image[:, :, 2]
        
        ch1_substracted = ch1 - np.mean(ch1)
        ch2_substracted = ch2 - np.mean(ch2)
        ch3_sunstracted = ch3 - np.mean(ch3)
        
        return np.stack((ch1_substracted, ch2_substracted, ch3_substracted), axis =0)
    
class MakeTensor:
    def __call__(self, obj):
        return torch.from_numpy(obj)

shared_transform = transforms.Compose(
[Resize(32, 32),
SubtractMean(),
MakeTensor()
])

In [7]:
def lines(file_path, encoding = "utf-8"):
    with open(file_path, "rt", encoding = encoding) as f:
        for line in f:
            yield line.rstrip("\n")
            

In [18]:
def read_annotation(annotation_path):
    data = []
    for line in lines(annotation_path):
        cells = line.split(";")
        if cells[0] == "Filename":
            continue
        image_path, klass = os.path.join(os.path.dirname(annotation_path), cells[0]), int(cells[-1])
        data.append((image_path, klass))
    return data

In [32]:
#os.listdir('train')

In [20]:
class_dirs = [o for o in os.listdir('train') ]

In [22]:
num_classes = len(class_dirs)

In [23]:
print(num_classes)

43


In [24]:
data = []

In [26]:
for d in class_dirs:
    csv_file = "GT-{}.csv".format(d)
    annotation_path = os.path.join('train', d, csv_file)
    data += read_annotation(annotation_path)

In [27]:
size = len(data)

In [28]:
size

39209

In [29]:
data[0]

('train/00023/00000_00000.ppm', 23)

In [30]:
image_path, klass = data[0]
sample = io.imread(image_path)

In [33]:
#sample

In [34]:
class GTRSB(Dataset):
    def __init__(self, transform = None):
        self.size = 0
        self.data = []
        self.transform = transform
    def __getitem__(self, index):
        image_path, klass = self.data[index]
        image_sample = io.imread(image_path)
        return image_sample, klass
    def __len__(self):
        return self.size
    
        
    

In [36]:
class GTRSBTrain(GTRSB):
    def __init__(self, path = 'train', transform = None):
        self.class_dirs = [fold for fold in os.listdir('train') ]
        self.num_classes = len(self.class_dirs)
        
        self.data = []
        
        for fold in self.class_dirs:
            csv = "GT-{}.csv".format(fold)
            annotation_path = os.path.join('train', fold, csv)
            self.data += read_annotation(annotation_path)
            
        self.size = len(self.data)

In [37]:
newobj = GTRSBTrain()

In [40]:
newobj.size

39209

In [42]:
#newobj.data

In [47]:
#CNN Architecture
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size = 3) # 32 filters: 3 x 3
        self.conv2 = nn.Conv2d(32, 32, kernel_size = 3) # 32 filters: 3 x 3

        self.conv3 = nn.Conv2d(32, 64, kernel_size = 3) # 64 filters: 3 x 3
        self.conv4 = nn.Conv2d(64, 64, kernel_size = 3) # 64 filters: 3 x 3
        self.conv5 = nn.Conv2d(64, 64, kernel_size = 3) # 64 filters: 3 x 3

        self.linear1 = nn.Linear(64 * 4 * 4, 512)
        self.linear2 = nn.Linear(512, 43)

    def forward(self, x):
        # Conv, ReLU, Conv, ReLU, Max-pooling
        relu1 = F.relu(self.conv1(x.float()))
        relu2 = F.relu(self.conv2(relu1))
        mp1 = F.max_pool2d(relu2, kernel_size = 2, stride = 2)

        # Conv, ReLU, Conv, ReLU, Max-pooling
        relu3 = F.relu(self.conv3(mp1))
        relu4 = F.relu(self.conv4(relu3))
        relu5 = F.relu(self.conv5(relu4))
        mp2 = F.max_pool2d(relu5, kernel_size = 2, stride = 2)

        # Fully-connected layer
        flat = mp2.view(mp2.size(0), -1)
        hidden = F.relu(self.linear1(flat))
        dropout = F.dropout(hidden)
        y = F.log_softmax(self.linear2(dropout), dim = 1)

        return y

In [48]:
#splitting train into valid (20%) and train(80%)

In [49]:
learning_rate = 1e-4
sgd_momentum = 0.5
epochs = 20
trainset = GTRSBTrain()

In [50]:
print('Number of samples: ', len(trainset))

Number of samples:  39209


In [56]:
def get_train_valid_loader(data,
                           batch_size = 4,
                           valid_size=0.2,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg


    num_train = len(data)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = DataLoader(data, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    return train_loader, valid_loader


In [57]:
train_loader, valid_loader = get_train_valid_loader(trainset)

In [59]:
len(train_loader)

7842

In [60]:
len(valid_loader)

1961

In [61]:
len(data)

39209